## Reading Tweets from msca-bdp-tweets bucket

### Using PySpark kernel

In [7]:
import os
import subprocess
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.types import StructType

import findspark
findspark.init()
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext


In [8]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [9]:
directory = 'gs://msca-bdp-tweets/final_project/'
# file = '*.json'
# path = directory + file
path = directory

In [8]:
cmd = 'hadoop fs -du -s -h ' + directory

p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
for line in p.stdout.readlines():
    print (line)
    
retval = p.wait()

617.8 G  617.8 G  gs://msca-bdp-tweets/final_project



In [10]:
%%time

tweets_df = spark.read.json(path)

22/03/16 19:23:49 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 36 for reason Container marked as failed: container_1647384369859_0003_01_000036 on host: hub-msca-bdp-dphub-students-bhadri-sw-750r.c.msca-bdp-students.internal. Exit status: -100. Diagnostics: Container released on a *lost* node.
22/03/16 19:23:49 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 35 for reason Container marked as failed: container_1647384369859_0003_01_000035 on host: hub-msca-bdp-dphub-students-bhadri-sw-750r.c.msca-bdp-students.internal. Exit status: -100. Diagnostics: Container released on a *lost* node.
22/03/16 19:23:49 ERROR org.apache.spark.scheduler.cluster.YarnScheduler: Lost executor 36 on hub-msca-bdp-dphub-students-bhadri-sw-750r.c.msca-bdp-students.internal: Container marked as failed: container_1647384369859_0003_01_000036 on host: hub-msca-bdp-dp

CPU times: user 2.71 s, sys: 883 ms, total: 3.59 s
Wall time: 12min 1s


In [ ]:
%%time

tweets_df.count()

CPU times: user 1.13 s, sys: 335 ms, total: 1.46 s
Wall time: 4min 46s


100011000

In [8]:
bucket_write = 'msca-bdp-students-bucket/shared_data/bhadri/tweets'

In [11]:
file1_path = 'gs://msca-bdp-students-bucket/shared_data/bhadri/tweets50k.csv'
#load csv to df
twr50k_df = spark.read.csv(file1_path, header='true', inferSchema='true', sep=',', quote='"')
# twr50k_df.cache()

In [16]:
test_pdf = twr50k_df.limit(100).toPandas()

In [20]:
twr50k_df.limit(3)

_c0,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,place,possibly_sensitive,quote_count,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,reply_count,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user,withheld_in_countries
0,null,null,Sun Jan 16 20:55:...,null,"Row(hashtags=[], ...",null,null,0,False,low,null,1482818815730008064,1482818815730008064,null,null,null,null,null,False,en,null,False,0,null,null,null,null,0,0,False,"""Row(contributors...",text='COVID 🦠 A...,truncated=False,user=Row(contrib...,created_at='Thu ...,default_profile=...,default_profile_...
1,null,null,Sun Jan 16 20:55:...,null,"Row(hashtags=[], ...",null,"""Row(display_text...",0,False,low,null,1482818815960526850,1482818815960526850,null,null,null,null,null,False,en,null,False,0,null,null,null,null,0,0,False,null,"""<a href=""""https:...",Maryland Gov. Hog...,1642366529823,True,Row(contributors_...,null
2,null,null,Sun Jan 16 20:55:...,null,"Row(hashtags=[], ...",null,"""Row(display_text...",at least in NJ. ...,000 at the peak to 2,400 yesterday. \...,0,False,low,null,1482818816027541510,1482818816027541510,null,null,null,null,null,True,en,null,null,0,"""Row(contributors...",text='Do the Omi...,truncated=False,user=Row(contrib...,created_at='Sat ...,default_profile=...,default_profile_...,description='44 ...,Bears,Bulls,Black Hawks


In [17]:
# test_pdf.to_csv('gs://msca-bdp-students-bucket/shared_data/bhadri/tweets100.csv')

In [21]:
sample_df = tweets_df.limit(1000000)

In [23]:
sample_df.count()

1000000

In [25]:
sample_df.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- embeddable: boolean (nullable = true)
 |    |    |    |    |-- monetizable: bo

In [27]:
sample_df = sample_df.select(['user','text','coordinates','quote_count','reply_count','retweet_count'])

ModuleNotFoundError: No module named 'spark'

In [28]:
sample_df.select('lang').distinct().show()

+----+
|lang|
+----+
|  en|
+----+



In [48]:
print(twr50k_df.schema)

StructType(List(StructField(_c0,StringType,true),StructField(contributors,StringType,true),StructField(coordinates,StringType,true),StructField(created_at,StringType,true),StructField(display_text_range,StringType,true),StructField(entities,StringType,true),StructField(extended_entities,StringType,true),StructField(extended_tweet,StringType,true),StructField(favorite_count,StringType,true),StructField(favorited,StringType,true),StructField(filter_level,StringType,true),StructField(geo,StringType,true),StructField(id,StringType,true),StructField(id_str,StringType,true),StructField(in_reply_to_screen_name,StringType,true),StructField(in_reply_to_status_id,StringType,true),StructField(in_reply_to_status_id_str,StringType,true),StructField(in_reply_to_user_id,StringType,true),StructField(in_reply_to_user_id_str,StringType,true),StructField(is_quote_status,StringType,true),StructField(lang,StringType,true),StructField(place,StringType,true),StructField(possibly_sensitive,StringType,true),St

In [52]:
print(tweets_df.schema("user").dataType)

TypeError: 'StructType' object is not callable

In [34]:
twr50k_df.columns

['_c0',
 'contributors',
 'coordinates',
 'created_at',
 'display_text_range',
 'entities',
 'extended_entities',
 'extended_tweet',
 'favorite_count',
 'favorited',
 'filter_level',
 'geo',
 'id',
 'id_str',
 'in_reply_to_screen_name',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'is_quote_status',
 'lang',
 'place',
 'possibly_sensitive',
 'quote_count',
 'quoted_status',
 'quoted_status_id',
 'quoted_status_id_str',
 'quoted_status_permalink',
 'reply_count',
 'retweet_count',
 'retweeted',
 'retweeted_status',
 'source',
 'text',
 'timestamp_ms',
 'truncated',
 'user',
 'withheld_in_countries']

In [20]:
#null in imp cols
col_list = ['user','text']
col_list2 = ['text']

In [59]:
from pyspark.sql.functions import col,isnan, when, count
twr50k_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in col_list]).show()

+-----+-----+
| user| text|
+-----+-----+
|15088|13832|
+-----+-----+



In [23]:
from pyspark.sql.functions import col,isnan, when, count
covid_df.select([count(when(col(c).isNull(), c)).alias(c) for c in col_list]).show()

+----+----+
|user|text|
+----+----+
|   0|   0|
+----+----+



In [41]:
from pyspark.sql.functions import col
sample_df = tweets_df.select(col("id"),col("id_str"),col("user"),col("place"),col("text"),col("entities"),col("created_at"), col("coordinates"),col("favorite_count"),col("quote_count"),col("retweet_count"),col("reply_count"),col("is_quote_status"),col("retweeted"),col("lang"))


In [25]:
import pyspark.sql.functions as sql_fun
covid_df = sample_df.filter(sql_fun.lower(sample_df.text).contains('covid'))
covid_df.count()

22/03/16 21:52:43 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 150 for reason Container marked as failed: container_1647384369859_0003_01_000151 on host: hub-msca-bdp-dphub-students-bhadri-sw-91jh.c.msca-bdp-students.internal. Exit status: -100. Diagnostics: Container released on a *lost* node.
22/03/16 21:52:43 ERROR org.apache.spark.scheduler.cluster.YarnScheduler: Lost executor 149 on hub-msca-bdp-dphub-students-bhadri-sw-91jh.c.msca-bdp-students.internal: Container marked as failed: container_1647384369859_0003_01_000150 on host: hub-msca-bdp-dphub-students-bhadri-sw-91jh.c.msca-bdp-students.internal. Exit status: -100. Diagnostics: Container released on a *lost* node.
22/03/16 21:52:43 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 149 for reason Container marked as failed: container_1647384369859_0003_01_000150 on host: hub-msca-bdp

61719499

In [ ]:
from pyspark.sql.functions import col,isnan, when, count
covid_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in col_list]).show()

In [16]:
import pyspark.sql.functions as sql_fun
corona_df = sample_df.filter(sql_fun.lower(sample_df.text).contains('corona'))
corona_df.count()

806681

In [18]:
corona_df.write.mode("overwrite").parquet("gs://msca-bdp-students-bucket/shared_data/bhadri/corona_df.parquet")

In [19]:
!gsutil du -sh -a gs://msca-bdp-students-bucket/shared_data/bhadri/corona_df.parquet

1338225028   gs://msca-bdp-students-bucket/shared_data/bhadri/corona_df.parquet


In [26]:
corona_pq_df = spark.read.parquet("gs://msca-bdp-students-bucket/shared_data/bhadri/corona_df.parquet")

In [27]:
corona_pq_df.count()

806681

In [34]:
corona_pq_df.filter(~ ((corona_pq_df.retweeted == True) | (corona_pq_df.retweeted == False)) ).count()

0

In [33]:
corona_pq_df.filter(~ ((corona_pq_df.is_quote_status == True) | (corona_pq_df.is_quote_status == False)) ).count()

0

In [36]:
corona_pq_df.filter(corona_pq_df.retweeted.isNull()).count()

0

In [38]:
corona_pq_df = corona_pq_df.withColumn('type',(when(corona_pq_df.is_quote_status, 'quote').when(corona_pq_df.retweeted, 'retweet').otherwise('tweet')))

In [39]:
corona_pq_df.limit(10)

id,id_str,user,text,entities,created_at,coordinates,favorite_count,quote_count,retweet_count,reply_count,is_quote_status,retweeted,lang,type
1462399046233083909,1462399046233083909,"{false, Sat Apr 0...",RT @drsimonegold:...,"{[], null, [], []...",Sun Nov 21 12:34:...,null,0,0,0,0,false,false,en,tweet
1462399048284094466,1462399048284094466,"{false, Wed Jul 2...",RT @drsimonegold:...,"{[], null, [], []...",Sun Nov 21 12:34:...,null,0,0,0,0,false,false,en,tweet
1462399058081845252,1462399058081845252,"{false, Thu Nov 2...",RT @AJEnglish: “T...,"{[], null, [], []...",Sun Nov 21 12:34:...,null,0,0,0,0,false,false,en,tweet
1462399079489720324,1462399079489720324,"{false, Sun Mar 2...",RT @drsimonegold:...,"{[], null, [], []...",Sun Nov 21 12:34:...,null,0,0,0,0,false,false,en,tweet
1462399106421243913,1462399106421243913,"{false, Tue Jun 2...",RT @drsimonegold:...,"{[], null, [], []...",Sun Nov 21 12:34:...,null,0,0,0,0,false,false,en,tweet
1462399113631313924,1462399113631313924,"{false, Mon Feb 0...",RT @AJEnglish: “T...,"{[], null, [], []...",Sun Nov 21 12:34:...,null,0,0,0,0,false,false,en,tweet
1462399114877022209,1462399114877022209,"{false, Wed May 2...",Covid-19 and Car ...,"{[], null, [], [{...",Sun Nov 21 12:34:...,null,0,0,0,0,false,false,en,tweet
1462399201183105028,1462399201183105028,"{false, Sun Jan 3...",The COVID-19 Pand...,"{[], null, [], [{...",Sun Nov 21 12:35:...,null,0,0,0,0,false,false,en,tweet
1462399222834212867,1462399222834212867,"{false, Sun Jul 3...",RT @dwnews: Coron...,"{[], null, [], []...",Sun Nov 21 12:35:...,null,0,0,0,0,false,false,en,tweet
1462399224004415501,1462399224004415501,"{false, Wed Jul 2...",RT @drsimonegold:...,"{[], null, [], []...",Sun Nov 21 12:35:...,null,0,0,0,0,false,false,en,tweet


In [40]:
corona_pq_df.groupBy("type") \
        .agg(count(corona_pq_df.type)) \
        .sort(desc("count(type)"))

type,count(type)
tweet,737401
quote,69280
